In [4]:
# ----------------------------------------------------------------------
# A widget that allows you to choose server-side files or directories.
# ======================================================================
#  AUTHOR: Rob Campbell, Purdue University
#  Copyright (c) 2004-2017  HUBzero Foundation, LLC
#  See LICENSE file for details.
# ======================================================================
import os
import ipywidgets as ui
from IPython.display import display

class PathSelector():

    def __init__(self,start_dir,select_file=True,single=True):
        self.file        = None 
        self.select_file = select_file
        self.value       = start_dir
        self.single      = single
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 
        
        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.value)
        self.select.observe(self.on_update,'value')
            
    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][-1])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.value,item))
        
        if os.path.isfile(path):
            if self.select_file:
                
                # Clear other selections if in single mode
                if self.single and len(self.select.value) > 1:
                    save = self.select.value[-1]
                    with self.select.hold_trait_notifications():
                        self.select.value = ()
                    self.select.value = (save,) 
                
                # Update title and collapse
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # os.path.isdir(path)
            self.file = None 
            self.value  = path
            
            # Build list of files and dirs
            keys = ['[..]']; 
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()

    def _ipython_display_(self):
        self.accord._ipython_display_()
        
    @property
    def disabled(self):
        return self.select.disabled

    @disabled.setter
    def disabled(self, newval):
        self.select.disabled = newval

    @property
    def visible(self):
        return self.accord.layout.visibility

    @visible.setter
    def visible(self, newval):
        if newval:
            self.accord.layout.visibility = 'visible'
            return
        self.accord.layout.visibility = 'hidden'

In [5]:
f = PathSelector('/')
f

Accordion(children=(SelectMultiple(options=(('$WINRE_BACKUP_PARTITION.MARKER', '$WINRE_BACKUP_PARTITION.MARKER…

In [6]:
f.value


'C:\\'

In [143]:
f.visible=False

In [9]:
f.visible

In [10]:
f.visible=True

In [13]:
f.select.value

()

In [14]:
x = (1,)
type(x)

tuple